In [1]:
import fiona
import os
import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
from ipywidgets import HTML
from IPython.display import clear_output
from ipyleaflet import Map, basemaps, basemap_to_tiles, CircleMarker, Popup, Polyline, Polygon, LayerGroup, LayersControl,ScaleControl, FullScreenControl
from ipyleaflet import WidgetControl
from ipywidgets import interactive
import matplotlib as mpl
import ipywidgets as widgets
from ipywidgets import HBox, Label, IntSlider,IntRangeSlider
#import matplotlib.patches as patches
#from matplotlib.patches import Polygon
import matplotlib
import matplotlib.colors
import contextily as cx
import shapely.geometry as sg
from rijksdriehoek import rijksdriehoek
from matplotlib.collections import PatchCollection
import ipympl
import warnings
import numpy as np
import ast
import datetime as dt


RD = rijksdriehoek.Rijksdriehoek()
plt.rcParams.update({'font.size': 24})

In [2]:
def read_gpkg(filename):
    
    assert '.' in filename, f'Filename {filename} does not have an extension!'    
    if filename.split('.')[1] != 'gpkg':
        raise ValueError(f'Expected gpkg, got {filename}')
    if not os.path.isfile(f'Data/{filename}'):
        raise ValueError(f'File {filename} not found in Data directory!')
    
    AOI = {}
    with fiona.open(f"Data/{filename}") as layer:
        for feature in layer:     
            AOI[feature['properties']['ogc_fid']] = {
                "crd_dec": [feature['geometry']['coordinates'][ii] for ii in range(len(feature['geometry']['coordinates']))]
                }
            for key in ['h0', 's_h0', 'xI', 's_xI', 'xP', 's_xP', 'xE', 's_xE', 'tau', 's_tau',
                        'rho_xIh0', 'rho_xPh0', 'rho_xEh0', 'rho_tauh0',
                        'rho_xPxI', 'rho_xExI', 'rho_xItau', 
                        'rho_xPxE', 'rho_xPtau', 
                        'rho_xEtau', 'xI_frac']:
                AOI[feature['properties']['ogc_fid']][key] = feature['properties'][key]
    return AOI

def convert_wgs84_to_rd(list_of_coords):
    """
    Converts a list of coordinates in WGS84 to RD
    :param list_of_coords: list of (lon, lat) objects
    :return: list of (rd_x, rd_y) objects
    """
    rd_coords = []
    for coord in list_of_coords:
        RD.from_wgs(lon=coord[0], lat=coord[1])
        rd_coords.append([RD.rd_x, RD.rd_y])
    return rd_coords

def convert_rd_to_wgs84(list_of_coords):
    """
    Converts a list of coordinates in RD to WGS
    :param list_of_coords: list of (rdx, rdy) objects
    :return: list of (lon, lat) objects
    """
    wgs_coords = []
    for coord in list_of_coords:
        RD.rd_x = coord[0]
        RD.rd_y = coord[1]
        wgs_coords.append(RD.to_wgs())
    return wgs_coords

def get_yearfrac(date):
    year = eval(date[:4])
    month = eval(date[5:7].lstrip('0'))
    day = eval(date[8:].lstrip('0'))
    dayOfYear = dt.datetime(year=year, month=month, day=day) - dt.datetime(year=year, month=1, day=1)
    yearDays = dt.datetime(year=year+1, month=1, day=1) - dt.datetime(year=year, month=1, day=1)
    yearfrac = year + dayOfYear.days / yearDays.days
    return yearfrac

In [3]:
def read_aoi():
    filename = "Data/AoI_GreenHeart-polygon.shp"
    shape = fiona.open(filename)
    finished = False
    shape_iter = iter(shape)
    aoi_rd_poly = None
    while not finished:
        try:
            shp = next(shape_iter)
            polygons = [sg.Polygon(convert_wgs84_to_rd(shp["geometry"]["coordinates"][ii])) for ii in range(len(shp["geometry"]["coordinates"]))]
            if len(polygons) > 1 or aoi_rd_poly is not None:
                raise ValueError("Cannot handle AOI with more than 1 polygon!")
            aoi_rd_poly = polygons[0]
            if not aoi_rd_poly.is_valid:
                raise ValueError("Invalid polygon AOI!")
        except StopIteration:
            finished = True
    if aoi_rd_poly is None:
        raise ValueError(f"No valid AOI polygon found in {filename}!")
    return convert_rd_to_wgs84(np.array(aoi_rd_poly.exterior.coords.xy).T)
aoi_wgs_poly = read_aoi()

In [4]:
def read_KNMI_file(filename):
    f = open(filename)
    data = f.read().split('\n')
    f.close()
    start_idx = None
    for i in range(len(data)):
        if '# STN,YYYYMMDD' in data[i]:
            start_idx = i + 2
            break
    headers = data[start_idx-2].split(',')
    headers = [h.strip() for h in headers]
    evapo_id = headers.index('EV24')
    date_id = headers.index('YYYYMMDD')
    precip_id = headers.index('RH')
    temp_id = headers.index('TX')
    processed_data = {}
    for row in data[start_idx:]:
        if row != '':
            split_row = row.split(',')
            date = dt.datetime.strptime(split_row[date_id].strip(), '%Y%m%d')
            if '' not in [date, split_row[evapo_id].strip(), split_row[precip_id].strip(), split_row[temp_id].strip()]:
                if '348' in filename and date.year < 1988: # very incomplete data before 1988
                    continue
                processed_data[date] = [eval(split_row[precip_id].strip()),
                                        eval(split_row[evapo_id].strip()),
                                        eval(split_row[temp_id].strip())]
                if processed_data[date][0] == -1:
                    processed_data[date][0] = 0
                processed_data[date][0] /= 10
                processed_data[date][1] /= 10
                processed_data[date][2] /= 10
    return processed_data

In [5]:
gpkg_data = read_gpkg('InSAR_ALS_GNSS_Grav_Leveling.gpkg')

In [6]:
t0 = '2015-01-02'
Cabauw = read_KNMI_file(f'Data/etmgeg_348.txt')
tmax = max(Cabauw.keys())
tmax = '{:0>4d}-{:0>2d}-{:0>2d}'.format(tmax.year, tmax.month, tmax.day)
tmin = min(Cabauw.keys()) + dt.timedelta(days=365) # to give space for tau to run back
tmin = '{:0>4d}-{:0>2d}-{:0>2d}'.format(tmin.year, tmin.month, tmin.day)
etmgeg = Cabauw 

tmin_yf = get_yearfrac(tmin)
tmax_yf = get_yearfrac(tmax)
t0_yf = get_yearfrac(t0)
t0_dt = dt.datetime.strptime(t0, '%Y-%m-%d')
tmin_dt = dt.datetime.strptime(tmin, '%Y-%m-%d')
tmax_dt = dt.datetime.strptime(tmax, '%Y-%m-%d')

t_eval_min = dt.datetime(1960, 1, 1)

In [23]:
def get_data(file):
    EUP_f = open(f'Data/EUPs/{file}.txt')
    raw = EUP_f.read()
    EUP_f.close()
    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
    EUP = eval(raw_EUP)

    if EUP['InSAR'] != -999:
        for keykey in [k for k in EUP['InSAR'].keys() if k[0] == 'h']:
            for d in range(len(EUP['InSAR'][keykey])):
                if EUP['InSAR'][keykey][d] == 'nan':
                    EUP['InSAR'][keykey][d] = np.nan

    
    y = []
    y_insar = []
    t = []
    t_insar = []
    s = []
    s_insar = []
    min_t = None
    max_t = None

    if EUP['EUP_type'] == 'brp':
        for key in ['80_AHN1', '80_AHN2', '80_AHN3', '80_AHN4', '_ThMD']:
            if EUP[f't{key}'] not in [-999, '']:
                y.append(EUP[f'h{key}'])
                t.append(EUP[f't{key}'])
                s.append(max(EUP[f's{key}'], 0.001))

        if EUP['InSAR'] != -999:
            keys = [key[1:] for key in EUP['InSAR'].keys() if key[0] == 't']
            for key in keys:
                for t_ in range(len(EUP['InSAR'][f't{key}'])):
                    if not np.isnan(EUP['InSAR'][f'h{key}'][t_]):
                        t_insar.append(EUP['InSAR'][f't{key}'][t_])
                        y_insar.append(EUP['InSAR'][f'h{key}'][t_])
                        s_insar.append(0.01)

    else:
        for key in ['100_AHN1', '100_AHN2', '100_AHN3', '100_AHN4', '_ThMD']:
            if EUP[f't{key}'] not in [-999, '']:
                y.append(EUP[f'h{key}'])
                t.append(EUP[f't{key}'])
                s.append(max(EUP[f's{key}'], 0.001))
    full_y = []
    full_t = []
    full_s = []
    for d in range(len(y)):
        full_y.append(y[d])
        full_t.append(get_yearfrac(t[d]))
        full_s.append(s[d])
    for d in range(len(y_insar)):
        full_y.append(y_insar[d]+EUP['h0'])
        full_t.append(get_yearfrac(t_insar[d]))
        full_s.append(s_insar[d])
    return full_y, full_t, full_s

def plot_model(file):
    EUP_f = open(f'Data/EUPs/{file}.txt')
    raw = EUP_f.read()
    EUP_f.close()
    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
    EUP = eval(raw_EUP)

    if EUP['InSAR'] != -999:
        for keykey in [k for k in EUP['InSAR'].keys() if k[0] == 'h']:
            for d in range(len(EUP['InSAR'][keykey])):
                if EUP['InSAR'][keykey][d] == 'nan':
                    EUP['InSAR'][keykey][d] = np.nan
    tt = []
    h = []
    cur_t = t_eval_min
    while get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(cur_t.year, cur_t.month, cur_t.day)) < tmax_yf:
        tt.append('{:0>4d}-{:0>2d}-{:0>2d}'.format(cur_t.year, cur_t.month, cur_t.day))
        cur_t += dt.timedelta(days=1)

    xI = EUP['xI']
    xP = EUP['xP']
    xE = EUP['xE']
    h0 = EUP['h0']
    tau = EUP['tau']

    ndays = (tmax_dt-tmin_dt).days

    R_array = np.zeros((ndays, ))
    I_array = np.zeros((ndays, ))
    t_array = np.zeros((ndays, ))
    t0_idx = None

    for day in range(ndays):
        for backwards_day in range(int(tau)+1):
            cur_day = tmin_dt + dt.timedelta(days=day-backwards_day)
            try:
                Pt = etmgeg[cur_day][0]
                Et = etmgeg[cur_day][1]
                if backwards_day == int(tau):
                    R_array[day] += (Pt*xP - Et*xE) * (tau % 1)
                else:
                    R_array[day] += (Pt*xP - Et*xE)

            except KeyError:
                continue

        if R_array[day] <= 0:
            I_array[day] = I_array[day-1] + xI
        else:
            I_array[day] = I_array[day-1]

        date = tmin_dt + dt.timedelta(days=day)
        t_array[day] = get_yearfrac('{:0>4d}-{:0>2d}-{:0>2d}'.format(date.year, date.month, date.day))
        if date == t0_dt:
            t0_idx = day

    t_lst = list(t_array)
    if t0_idx is None:
        raise ValueError('t0 > tmax!')

    xIfrac = sum(1*R_array<=0)/R_array.shape[0]

    R_array -= R_array[t0_idx]
    I_array -= I_array[t0_idx]


    for t in tt:
        if get_yearfrac(t) < tmin_yf:
            elev = h0 + R_array[0] + I_array[0] + (dt.datetime.strptime(t, '%Y-%m-%d') - tmin_dt).days * xI * xIfrac
        else:
            tidx = t_lst.index(get_yearfrac(t))
            elev = h0 + R_array[tidx]+I_array[tidx]
        h.append(elev)

    return [get_yearfrac(t) for t in tt], h

def convert_value_to_hex(value, vmin=-10, vmax=10):
    pct = min(max((value - vmin) / (vmax - vmin), 0), 1)
    clr = (int(255 * min(1, 2 - 2 * pct)), 
           int(255 * (1 - 2 * abs(0.5 - pct))), 
           int(255 * min(1, 2 * pct)))
    #clr = (int(255 * max(0, (1 - 2 * pct))), 0, (int(255 * max(2*pct - 1, 0))))
    hx = '#'
    for c in clr:
        
        hx += "{:0>2s}".format(hex(c)[2:])
    return hx

def map_index(EUP):
    """
    This function maps the index of the gpkg EUP to the txt EUP, as these are not the same:
    the gpkg maintains the original numbering before ~50000 EUPs were removed, the txt EUP
    starts counting from 0 without gaps
    :param EUP: the gpkg index of the EUP
    :return: the txt index of the EUP
    """
    keys = list(gpkg_data.keys())
    return keys.index(EUP)


In [8]:
output = widgets.Output()

In [24]:
def plot_overview(plot='xI'):
    with output:
        clear_output(wait=True)
        display(main_plot)
    if plot != 'Select':
        vdir = {'xI': [-10, 10],
                's_xI': [0, 10],
                'h0': [-8, 2],
                's_h0': [0, 10]}
                    
        with output:
            CarDB = basemap_to_tiles(basemaps.CartoDB.Positron)
            CarDB.base = True
            CarDB.name = 'Map (CartoDB)'
        
            img_nl = basemap_to_tiles(basemaps.nlmaps.luchtfoto)
            img_nl.base = True
            img_nl.name = 'Imagery (nlmaps)'
            
            m = Map(center=list(np.mean(np.array(aoi_wgs_poly), axis=0)), zoom=9, close_popup_on_click=False , layers = [img_nl,CarDB])

            AoI = LayerGroup(name = 'AoI')
            EUPs = LayerGroup(name = 'EUPs')
        
            m.add_layer(AoI)
            m.add_layer(EUPs)

            line = Polyline(locations = aoi_wgs_poly,
                        weight = 2,
                        color= '#000000', linestyle='--', fill=False) 

            highlighter = Polyline(locations = [[0, 0], [0, 0]],
                                   weight=3,
                                   color='#5abf4d', fill=False)
            
            # line.on_click(create_button_click_arc(j))
            AoI.add_layer(line)
            m.add_control(LayersControl())

            

            def plot_data(EUP, m):
                def button_click(**kwargs):
                    txt_eup = map_index(EUP)
                    t, h = plot_model(txt_eup)
                    y, yt, ys = get_data(txt_eup)
                    EUP_f = open(f'Data/EUPs/{txt_eup}.txt')
                    raw = EUP_f.read()
                    EUP_f.close()
                    raw_EUP = raw.replace('nan', "'nan'").replace('array', 'np.array')
                    data = eval(raw_EUP)
                    m.remove_layer(m.layers[-1])
                    highlighter = Polyline(locations = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0]),
                                weight=3,
                                color='#5abf4d', fill=False)
                    m.add_layer(highlighter)
                    with out1:
                        clear_output(wait=True)
                        plt.figure(figsize=(11,5))
                        plt.plot(t, h, label='Model fit')
                        plt.errorbar(yt, y, yerr=ys, fmt='o', ecolor='r', label='Observations')
                        plt.grid(True)
                        plt.xlabel('Time [y]')
                        plt.ylabel('Elevation [m-NAP1997]')
                        plt.legend()
                        plt.show()
                    with out2:
                        clear_output(wait=True)
                        print(f'GPKG EUP {EUP} (TXT {txt_eup})')
                        print(f"Mean Yearly Irreversible Subsidence: {round(data['xI'] * data['xI_frac'] * 365.2425 * 1000, 1)}±{round(data['s_xI'] * data['xI_frac'] * 365.2425 * 1000, 1)} mm/y")
                        print(f"Elevation (1 Jan 2015): {round(data['h0'], 3)}±{round(data['s_h0'], 3)} m-NAP1997")
                return button_click
            
            for EUP in list(gpkg_data.keys())[:2000]:
                if plot in ['xI', 's_xI']:
                    val = gpkg_data[EUP][plot] * gpkg_data[EUP]['xI_frac'] * 365.2425 * 1000 # mm/y
                elif plot in ['h0']:
                    val = gpkg_data[EUP][plot] # m
                elif plot in ['s_h0']:
                    val = gpkg_data[EUP][plot] * 100 # cm
                else:
                    val = gpkg_data[EUP][plot] * 1000
                polygon = Polygon(
                    locations = convert_rd_to_wgs84(gpkg_data[EUP]['crd_dec'][0]),
                    weight=0,
                    fill_color=convert_value_to_hex(val, vdir[plot][0], vdir[plot][1]),
                    fill_opacity=1)
                polygon.on_click(plot_data(EUP, m=m))
                EUPs.add_layer(polygon)

            out1 = widgets.Output()
            out2 = widgets.Output()
        
            tab = widgets.Tab(children = [out1, out2])
            tab.set_title(0, 'Estimated Model')
            tab.set_title(1, 'Parameters')

            m.add(FullScreenControl())
            m.add_layer(highlighter)
            
            display(tab)
            display(m)

In [25]:
plot_drop = widgets.Dropdown(
        options=['Select', 'xI', 's_xI', 'h0', 's_h0'],
        value='Select',
        description='Parameter:',
        disabled=False,
    )
main_plot = interactive(plot_overview, plot=plot_drop)


In [26]:
%matplotlib ipympl
button = widgets.Button(description="Run Me!")


display(button, output)

def on_button_clicked(b):
    with output:
        clear_output()
        display(main_plot)

button.on_click(on_button_clicked)

Button(description='Run Me!', style=ButtonStyle())

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "interactive(children=(Dropdown(descrip…